In [9]:
## Mysql접속, investarDB에 Table 생성

import pymysql

conn = pymysql.connect(host='localhost', user='root', password='1111', db='investar0221', charset='utf8')
cur = conn.cursor()
a = "create table if not exists company_info (code varchar(20),company varchar(40),last_update date,primary key (code))"
cur.execute(a)
b = "create table if not exists daily_price (code varchar(20),date date,open bigint(20),\
high bigint(20),low bigint(20),close bigint(20),diff bigint(20),volume bigint(20),primary key (code, date))"
cur.execute(b)
conn.commit()
conn.close()

In [10]:
## KRX에서 데이터 읽어서 DF로 저장

import pandas as pd

url = 'http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13'
krx = pd.read_html(url, header=0)[0]
krx = krx[['종목코드','회사명']]
krx = krx.rename(columns={'종목코드':'code','회사명':'company'})
krx.code = krx.code.map('{:06d}'.format)

In [11]:
## 종목코드 company_info에 저장

from datetime import datetime

conn = pymysql.connect(host='localhost', user='root', password='1111', db='investar0221', charset='utf8')
cur = conn.cursor()
sql = "select max(last_update) from company_info"
cur.execute(sql)
rs = cur.fetchone()
today = datetime.today().strftime('%Y-%m-%d')
if rs[0] == None or str(rs[0]) < today:
    for i in range(len(krx)):
        code = krx.code.values[i]
        company = krx.company.values[i]
        sql = f"replace into company_info (code, company, last_update) values ('{code}','{company}','{today}')"
        cur.execute(sql)
conn.commit()
conn.close()

In [12]:
## 네이버에서 파싱

import requests
import bs4
from bs4 import BeautifulSoup

url = "http://finance.naver.com/item/sise_day.nhn?code=000210"
res = requests.get(url, headers={'User-agent':'Mozilla/5.0'})
soup = BeautifulSoup(res.text,'html.parser')
html = BeautifulSoup(requests.get(url,headers={'User-agent': 'Mozilla/5.0'}).text, 'html.parser') ## soup = html

In [13]:
## 맨끝 페이지수 읽기

pgrr = soup.find('td', {'class':'pgRR'}) 
s = str(pgrr.a['href']).split('=')
lastpage = int(s[-1])

In [40]:
## 1페이지부터 끝페이지까지 읽어서 DF에 넣기

df = pd.DataFrame() 
company = '회사'
code = '000210'
for page in range(1, 6): ## (5 대신 lastpage+1)
    pg_url='http://finance.naver.com/item/sise_day.nhn?code={}&page={}'.format(code,page)
    df = df.append(pd.read_html(requests.get(pg_url,headers={'User-agent': 'Mozilla/5.0'}).text)[0])
    timenow = datetime.now().strftime('%Y-%m-%d %H:%M')
    print('[{}] {} ({}) : {:04d}/{:04d} pages are downloading...'.format(timenow, company, code, page, lastpage), end="\r")

In [43]:
## DF파일 전처리 작업

df = df.rename(columns={'날짜':'date','종가':'close','전일비':'diff','시가':'open','고가':'high','저가':'low','거래량':'volume'})
df['date']=df['date'].str.replace('.', '-')
df = df.dropna() ## NaN값 삭제
df[['close', 'diff', 'open', 'high', 'low', 'volume']] = df[['close','diff', 'open', 'high', 'low', 'volume']].astype(int) ## 정수로
df = df[['date', 'open', 'high', 'low', 'close', 'diff', 'volume']] ## DF 컬럼 순서 변경

C:\Users\User\AppData\Local\Temp/ipykernel_14136/3307289217.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['date']=df['date'].str.replace('.', '-')


In [46]:
## DF파일 daily_price 에 저장

conn = pymysql.connect(host='localhost', user='root', password='1111', db='investar0221', charset='utf8')
cur = conn.cursor()
for r in df.itertuples():
    sql = f"replace into daily_price values ('{code}','{r.date}',{r.open},{r.high},{r.low},{r.close},{r.diff},{r.volume})"
    cur.execute(sql)
conn.commit()
conn.close()